[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=Withdraw.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=Withdraw.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Withdraw

This code will connect to the _devnet_ cluster and withdraw 0.1 SOL from a Mango `Account`.

Unlike `deposit()`, you specify an address as the destination. You can withdraw from your Mango `Account` to anyone's wallet (be careful!). Also, you can withdraw tokens even if you don't own them (as long as you have sufficient collateral) by borrowing them - `withdraw()`'s `allow_borrow` boolean allows or prohibits borrowing tokens the `Account` doesn't own.

In [ ]:
import decimal
import mango

from solana.publickey import PublicKey


# Use our hard-coded devnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file. This wallet
# has a Mango Account at HhepjyhSzvVP7kivdgJH9bj32tZFncqKUwWidS1ja4xL.
wallet = mango.Wallet(bytes([67,218,68,118,140,171,228,222,8,29,48,61,255,114,49,226,239,89,151,110,29,136,149,118,97,189,163,8,23,88,246,35,187,241,107,226,47,155,40,162,3,222,98,203,176,230,34,49,45,8,253,77,136,241,34,4,80,227,234,174,103,11,124,146]))

with mango.ContextBuilder.build(cluster_name="devnet") as context:
    group = mango.Group.load(context)
    account = mango.Account.load(context, PublicKey("HhepjyhSzvVP7kivdgJH9bj32tZFncqKUwWidS1ja4xL"), group)
    sol_token = mango.token(context, "SOL")

    print("Wrapped SOL in account", account.slot_by_instrument(sol_token).net_value)

    withdraw_value = mango.InstrumentValue(sol_token, decimal.Decimal("0.1"))
    withdrawal_signatures = account.withdraw(context, wallet, wallet.address, withdraw_value, False)
    print("Waiting for withdraw transaction to confirm...", withdrawal_signatures)
    mango.WebSocketTransactionMonitor.wait_for_all(
            context.client.cluster_ws_url, withdrawal_signatures, commitment="processed"
        )

    account = mango.Account.load(context, account.address, group)
    print("Wrapped SOL after withdrawal", account.slot_by_instrument(sol_token).net_value)

print("Example complete.")